In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import csv

from Generator import Generator
from Normalizer import Normalizer
from RealData import RealData

import matplotlib.pyplot as plt
np.set_printoptions(suppress = True)

In [2]:
run = 9

In [3]:
def save(model, recipe_normalizer, quant_normalizer, accs):
    model.save(f"./models/{run}_rank_{rank}/{run}_rank_{rank}.h5")
    
    np.savetxt(f"./models/{run}_rank_{rank}/norm_recipe_max", recipe_normalizer.feature_max)
    np.savetxt(f"./models/{run}_rank_{rank}/norm_recipe_min", recipe_normalizer.feature_min)
    
    np.savetxt(f"./models/{run}_rank_{rank}/norm_quant_max", quant_normalizer.feature_max)
    np.savetxt(f"./models/{run}_rank_{rank}/norm_quant_min", quant_normalizer.feature_min)
    
    
    # open file for writing, "w" is writing
    w = csv.writer(open(f"./models/{run}_rank_{rank}/accuracy.csv", "w"))

    # loop over dictionary keys and values
    for key, val in accs.items():

        # write every key and value to file
        w.writerow([key, val])

## Parameters

#### Data Generation

In [4]:
rank = True
normalize = True

train_examples = 100000
test_examples = 100

#### Model

In [5]:
embed_dim = 1024                          
stack = 6                                    

optimizer = keras.optimizers.Nadam()        
loss = keras.losses.MSE


metrics = keras.metrics.MeanSquaredError()

2022-05-01 10:25:40.076804: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-01 10:25:40.171338: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-01 10:25:40.171812: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-01 10:25:40.173143: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

#### Training

In [6]:
batch_size = 128
epochs = 1000

validation_split = 0.2
verbose = 2

## Generate Data

In [ ]:
generator = Generator('nutrients.csv') # filename with data
generator.generate(train_examples) # generates arg number of examples

In [ ]:
if rank:
    generator.rank()
    
recipes = generator.recipes
quant = generator.quant
    
if normalize:
    recipe_normalizer = Normalizer()
    recipe_normalizer.fit(recipes)
    recipes = recipe_normalizer.normalize(recipes)
    
    quant_normalizer = Normalizer()
    quant_normalizer.fit(quant)
    quant = quant_normalizer.normalize(quant)
    

## Build Model

In [ ]:
embed_dim = embed_dim
stack = stack
i1 = keras.layers.Input(generator.recipes.shape[1:])
f1 = keras.layers.Flatten()(i1)
y1 = keras.layers.Dense(embed_dim)(f1)

# Stack residual blocks (skip connections)
for i in range(stack):
    y2 = y1
    y1 = keras.layers.Dense(embed_dim,
        activation=keras.activations.relu)(y1)
    y1 = keras.layers.Dropout(0.35)(y1)
    y1 = keras.layers.Add()([y1,y2])
    y1 = keras.layers.LayerNormalization()(y1)
    
        

if normalize:
    o1 = keras.layers.Dense(quant.shape[1],activation=keras.activations.softplus)(y1)
    
else:
    o1 = keras.layers.Dense(quant.shape[1])(y1)
    
model = keras.Model(i1,o1)

In [ ]:
#keras.utils.plot_model(model,to_file='recipe_model.png',show_shapes=True,expand_nested=True)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28)]         0           []                               
                                                                                                  
 flatten (Flatten)              (None, 28)           0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 1024)         29696       ['flatten[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 1024)         1049600     ['dense[0][0]']                  
                                                                                              

In [ ]:
model.compile(optimizer=optimizer,
            loss=loss,
            metrics=metrics)

In [ ]:
filepath = f"./models/{run}_rank_{rank}.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min', save_freq=250)
callbacks_list = [checkpoint]

## Train

In [ ]:
history = model.fit(
    recipes, quant, 
    batch_size=batch_size, 
    epochs=epochs, 
    validation_split = validation_split, 
    verbose=verbose,
    callbacks = callbacks_list
)

Epoch 1/1000


2022-05-01 10:25:51.976677: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-05-01 10:25:52.143755: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8301



Epoch 00001: loss improved from inf to 0.12345, saving model to ./models/9_rank_True.h5


/home/chandler/anaconda3/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Epoch 00001: loss improved from 0.12345 to 0.08611, saving model to ./models/9_rank_True.h5
625/625 - 12s - loss: 0.0725 - mean_squared_error: 0.0725 - val_loss: 0.0131 - val_mean_squared_error: 0.0131 - 12s/epoch - 19ms/step
Epoch 2/1000

Epoch 00002: loss improved from 0.08611 to 0.01582, saving model to ./models/9_rank_True.h5

Epoch 00002: loss improved from 0.01582 to 0.01476, saving model to ./models/9_rank_True.h5

Epoch 00002: loss improved from 0.01476 to 0.01393, saving model to ./models/9_rank_True.h5
625/625 - 9s - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.0110 - val_mean_squared_error: 0.0110 - 9s/epoch - 15ms/step
Epoch 3/1000

Epoch 00003: loss improved from 0.01393 to 0.01179, saving model to ./models/9_rank_True.h5

Epoch 00003: loss improved from 0.01179 to 0.01167, saving model to ./models/9_rank_True.h5
625/625 - 9s - loss: 0.0115 - mean_squared_error: 0.0115 - val_loss: 0.0141 - val_mean_squared_error: 0.0141 - 9s/epoch - 15ms/step
Epoch 4/1000

Epoc

In [ ]:
plt.figure(1)
# summarize history for accuracy
plt.subplot(211)
plt.plot(history.history['mean_squared_error'])
plt.plot(history.history['val_mean_squared_error'])
plt.plot(history.history['loss'])
plt.ylabel('Progress')
plt.xlabel('Epoch')
plt.legend(['Training','Validation'],loc='lower right')

## Accuracy Function Definition

In [ ]:
# accuracy formula:
#   measured values < 1 are set to 1
#   error = 100(( |Measured Value - Given Value| ) / max(Measured Value, Given Value))
#   accuracy = 100 - error
def accuracy(pred, actual):
    
    accs = {}
    totals = []
    
    for i in range(pred.shape[0]): # for each recipe
        recipe_accs = np.zeros((pred.shape[1])) # last element for accuracy across all ingrs
        recipe_tot = {}
        
        for j in range(pred.shape[1]): # for each ingredient
            measured_ingr = pred[i,j]
            given_ingr = actual[i,j]
            
            if given_ingr < 1:
                given_ingr = 1 # avoid divide by 0 error
            if measured_ingr < 1:
                measured_ingr = 1
            
            
            error = 100 * abs(measured_ingr - given_ingr)/max(given_ingr,measured_ingr)
            accuracy = round(abs(100 - error),2)
            
            
            if (accuracy < 0):
                accuracy = 0
            
            recipe_accs[j] = accuracy
        
        accs[f"{i}_mape"] = round((np.sum(recipe_accs) / recipe_accs.shape[0]),2)
        accs[f"{i}_ingrs"] = recipe_accs
        

        
    return accs

## Testing Fake Data

In [ ]:
test_gen = Generator('nutrients.csv')
test_gen.generate(test_examples) # generates arg number of examples

if rank:
    test_gen.rank()
    
test_recipes = test_gen.recipes
test_quant = test_gen.quant
    
if normalize:
    test_recipes = recipe_normalizer.normalize(test_recipes)
    test_quant = quant_normalizer.normalize(test_quant)

In [ ]:
pred = model.predict(test_recipes)
if normalize:
    pred = quant_normalizer.inv_normalize(pred)

In [ ]:
actual = test_gen.quant

## Testing Real Data

In [ ]:
recipe_quant = "./Recipe Data.csv"
ingr_nutr = "./nutrients.csv"

In [ ]:
realdata = RealData(recipe_quant,ingr_nutr)

In [ ]:
if rank:
    realdata.rank()

real_recipes = realdata.recipes
real_quant = realdata.quant
    
if normalize:
    real_recipes = recipe_normalizer.normalize(real_recipes)

In [ ]:
pred = model.predict(real_recipes)
if normalize:
    pred = quant_normalizer.inv_normalize(pred)

In [ ]:
actual = realdata.quant

In [ ]:
index = 6
for i in range(len(pred[0])):
    pred_num = str(round(pred[index,i],3))
    act_num = str(round(actual[index,i],3))
    
    print(f"pred: {pred_num} | actual: {act_num}" )

In [ ]:
np.set_printoptions(suppress = True)
accs = accuracy(pred,actual)

In [ ]:
save(model,recipe_normalizer,quant_normalizer, accs)